In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import read_csv
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from tcn import TCN, tcn_full_summary
import predict

batch_size, time_steps, input_dim = 32, 5, 9
dataset=pd.read_csv('SolarRadiationPrediction.csv',engine='python',nrows=576*15)
dataset=dataset.drop("Data",axis=1)
dataset=dataset.drop("Time",axis=1)
dataset.head(5)
dataset=dataset.values

dataset=dataset.astype('float32')

scalar1=MinMaxScaler(feature_range=(0,1))
scalar2=MinMaxScaler(feature_range=(0,1))
scalar_dim=dataset[:,1]
dataset=scalar1.fit_transform(dataset)

scalar_dim=scalar_dim.reshape(len(dataset),1)
scalar_dim=scalar2.fit_transform(scalar_dim)

In [17]:

train_size=int(len(dataset)*0.67)
test_size=len(dataset)- train_size
train,test=dataset[0:train_size,:],dataset[train_size:len(dataset),:]

def create_dataset(dataset,look_back):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-1):
        #a=dataset第0行i~i+look_back個
        a=dataset[i:(i+look_back),0:9]
        dataX.append(a)
        dataY.append(dataset[i+look_back,1])
    return np.array(dataX),np.array(dataY)

look_back=5
trainX,trainY=create_dataset(train,look_back)
testX,testY=create_dataset(test,look_back)
#tcn_layer = TCN(nb_filters=64,kernel_size=3,dropout_rate=0.0,padding='causal',activation='relu',input_shape=(time_steps, input_dim))
tcn_layer = TCN(nb_filters=110,
                kernel_size=10,
                dropout_rate=0.4331163281069671,
                activation='relu',padding='causal',
                nb_stacks=1,
                dilations=(1, 2, 4),
                input_shape=(time_steps, input_dim))
print('Receptive field size =', tcn_layer.receptive_field)

model = Sequential([tcn_layer,Dense(1)])
model.compile(optimizer='adam', loss='mse')
tcn_full_summary(model, expand_residual_blocks=False)
tcn_full_summary(model, expand_residual_blocks=False)

Receptive field size = 127
Use tensorboard instead. Example in keras-tcn/tasks/tcn_tensorboard.py.
Use tensorboard instead. Example in keras-tcn/tasks/tcn_tensorboard.py.


In [18]:
loss_arr=[]
epochs=30
best_MAE_pre=[]
best_MAE=10000
for i in range(epochs):
    history=model.fit(trainX, trainY, epochs=1, validation_split=0.2)
    loss_arr.append(history.history['loss'][0])
    trainPre=model.predict(trainX)
    testPre=model.predict(testX)

    trainPre=scalar2.inverse_transform(trainPre)
    trainY_ord=scalar2.inverse_transform([trainY])
    testPre=scalar2.inverse_transform(testPre)
    testY_ord=scalar2.inverse_transform([testY])

    testScore=math.sqrt(mean_squared_error(testY_ord[0,:],testPre[:,0]))
    print('Test Score:%.2f RMSE'%(testScore))
    MAE=mean_absolute_error(testY_ord[0,:],testPre[:,0])
    print('Test Score:%.2f MAE'%(MAE))


    if MAE<best_MAE:
        best_MAE=MAE
        best_MAE_pre=testPre
        print("best")

    print("best MAE: %.2f MAE"%(mean_absolute_error(testY_ord[0,:],best_MAE_pre[:,0])))
  

89/89 [==============================] - 1s 9ms/step
Test Score:191.73 RMSE
Test Score:112.81 MAE
best
best MAE: 112.81 MAE
89/89 [==============================] - 1s 7ms/step
Test Score:130.77 RMSE
Test Score:83.26 MAE
best
best MAE: 83.26 MAE
89/89 [==============================] - 1s 8ms/step
Test Score:145.87 RMSE
Test Score:86.11 MAE
best MAE: 83.26 MAE
89/89 [==============================] - 1s 9ms/step
Test Score:142.50 RMSE
Test Score:84.09 MAE
best MAE: 83.26 MAE
89/89 [==============================] - 1s 9ms/step
Test Score:128.54 RMSE
Test Score:74.84 MAE
best
best MAE: 74.84 MAE
89/89 [==============================] - 1s 10ms/step
Test Score:116.73 RMSE
Test Score:64.88 MAE
best
best MAE: 64.88 MAE
89/89 [==============================] - 1s 8ms/step
Test Score:127.37 RMSE
Test Score:71.39 MAE
best MAE: 64.88 MAE
89/89 [==============================] - 1s 11ms/step
Test Score:118.12 RMSE
Test Score:66.41 MAE
best MAE: 64.88 MAE
89/89 [==============================] -

In [19]:
# history=model.fit(trainX, trainY, epochs=1, validation_split=0.2)
# loss_arr.append(history.history['loss'][0])
# trainPre=model.predict(trainX)
# testPre=model.predict(testX)
# trainPre=scalar2.inverse_transform(trainPre)
# trainY_ord=scalar2.inverse_transform([trainY])
# testPre=scalar2.inverse_transform(testPre)
# testY_ord=scalar2.inverse_transform([testY])
# testScore=math.sqrt(mean_squared_error(testY_ord[0,:],testPre[:,0]))
# print('Test Score:%.2f RMSE'%(testScore))
# MAE=mean_absolute_error(testY_ord[0,:],testPre[:,0])
# print('Test Score:%.2f MAE'%(MAE))
# if MAE<best_MAE:
#     best_MAE=MAE
#     best_MAE_pre=testY_ord
#     print("best")
# print("best MAE: %.2f MAE"%(mean_absolute_error(best_MAE_pre[0,:],testPre[:,0])))
